import numpy as np
import pandas as pd
import gensim
import re

from konlpy.tag import Okt

from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import PerplexityMetric

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#데이터 불러오기
book_data = pd.read_csv(r"C:\Users\user\Desktop\졸업 프로젝트\인공지능(데이터,모델)\교보크롤링(소설).csv", encoding = "cp949")

print(len(book_data))

okt = Okt()

stop_word_path = r"C:\Users\user\Desktop\졸업 프로젝트\인공지능(데이터,모델)\stopword.txt"
with open(stop_word_path, "r", encoding='utf-8') as file:
    stop_words = file.read().splitlines()

In [23]:
#특수문자 숫자 제거
def remove_char_stopwords(text,stop_words):
    # 한국어 문자와 공백만 허용
    # 정규식으로 찾은 문자열을 ' '공백 처리
    text = re.sub(r'[^\w\s가-힣]', ' ', text)  # 특수 문자제거
    
    words = text.split()
    
    filtered_words = [word for word in words if len(word)>1]
    
    cleaned_text = ' '.join(filtered_words)

    # 원래의 공백을 복원하기 위해 공백을 원래대로 유지하는 로직 추가
    # 단어 사이의 공백을 원래 상태로 유지
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    extract_nouns = okt.nouns(cleaned_text)
    
    filtered_nouns= [word for word in extract_nouns if word not in stop_words]

    final_word_set = set(noun for noun in filtered_nouns if len(noun) > 1)
    
    LDA_data_set = list(final_word_set)
    
    return LDA_data_set

In [24]:
def perform_lda_analysis(documents, num_topics=5, num_top_words=10):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(documents)
    
    lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda.fit(X)
    
    all_words_set = set()
    
    for topic in lda.components_:
        top_words_idx = topic.argsort()[-num_top_words:][::-1]
        top_words = [vectorizer.get_feature_names_out()[index] for index in top_words_idx]
        all_words_set.update(top_words)
    
    return ','.join(all_words_set)

In [25]:
def add_recommended_words_column(df, stop_words):
    recommended_words = []
    for i in range(len(df)):
        data = df.iloc[i, 4]
        LDA_data = remove_char_stopwords(data, stop_words)
        final_data = perform_lda_analysis(LDA_data, num_topics=5, num_top_words=10)
        recommended_words.append(final_data)
    df['추천단어'] = recommended_words
    return df

In [26]:
# 데이터프레임에 새로운 열 추가
book_data_with_words = add_recommended_words_column(book_data, stop_words)

# 결과를 엑셀 파일로 저장
output_path = r"C:\Users\user\Desktop\졸업 프로젝트\인공지능(데이터,모델)\추천단어(LDA기반).csv"
book_data_with_words.to_csv(output_path, index=False, encoding='cp949')

print(f"엑셀 파일로 저장되었습니다: {output_path}")

엑셀 파일로 저장되었습니다: C:\Users\user\Desktop\졸업 프로젝트\인공지능(데이터,모델)\추천단어(LDA기반).csv


In [27]:
input_csv_path =r"C:\Users\user\Desktop\졸업 프로젝트\인공지능(데이터,모델)\추천단어(LDA기반).csv"

output_csv_path =r"C:\Users\user\Desktop\졸업 프로젝트\인공지능(데이터,모델)\추천단어_중복제거(LDA기반).csv"

# CSV 파일 읽어오기
book_data = pd.read_csv(input_csv_path, encoding="cp949")

# 중복 제거 (전체 데이터프레임 기준)
book_data_no_duplicates = book_data.drop_duplicates()

# 중복 제거된 결과를 새로운 CSV 파일로 저장
book_data_no_duplicates.to_csv(output_csv_path, index=False, encoding='cp949')

print(f"중복 제거된 CSV 파일이 저장되었습니다: {output_csv_path}")

중복 제거된 CSV 파일이 저장되었습니다: C:\Users\user\Desktop\졸업 프로젝트\인공지능(데이터,모델)\추천단어_중복제거(LDA기반).csv
